In [6]:
import sys, pkgutil, subprocess, json
print("Kernel interpreter:", sys.executable)

subprocess.check_call([sys.executable, "-m", "pip", "install", "-U",
                       "pip", "setuptools", "wheel"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "-U",
                       "geopandas", "folium", "branca", "pyproj", "shapely", "pyogrio"])

Kernel interpreter: /opt/anaconda3/bin/python
  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
  Attempting uninstall: wheel
    Found existing installation: wheel 0.44.0
    Uninstalling wheel-0.44.0:
      Successfully uninstalled wheel-0.44.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1 requires sympy==1.13.1; python_version >= "3.9", but you have sympy 1.13.2 which is incompatible.


  Using cached folium-0.20.0-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached branca-0.8.2-py3-none-any.whl.metadata (1.7 kB)
Using cached folium-0.20.0-py2.py3-none-any.whl (113 kB)
Using cached branca-0.8.2-py3-none-any.whl (26 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 24.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [folium]2m4/6 [geopandas]
Has geopandas? True


In [6]:
import os, json, math, webbrowser, re
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from branca.colormap import LinearColormap

# inputs
LR_CSV  = "/Users/reesemullen/Downloads/lr_metrics_by_community_area.csv"   # updated naming
RF_CSV  = "/Users/reesemullen/Downloads/rf_metrics_by_community_area.csv"
GBT_CSV = "/Users/reesemullen/Downloads/xgb_metrics_by_community_area.csv"

MODEL_LABELS = {
    LR_CSV:  "Logistic_Regression",
    RF_CSV:  "Random_Forest",
    GBT_CSV: "Gradient_Boosted_Trees",
}

# Chicago community areas shapefile
COMMUNITY_SHP = "/Users/reesemullen/Downloads/Boundaries - Community Areas_20251208/geo_export_0e4862d7-dcba-4d83-aafc-60ce367251d6.shp"
PER_MODEL_METRICS = ["FP_rate", "FN_rate"]
SINGLE_METRIC = "Arrest_Rate_true"

MIN_SUPPORT = 200
OUT_DIR = "maps_by_model_metric"
os.makedirs(OUT_DIR, exist_ok=True)

# normalization
def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    def norm(c):
        c = c.strip().lower()
        c = re.sub(r"[^0-9a-z]+", "_", c)
        c = re.sub(r"_+", "_", c).strip("_")
        return c
    return df.rename(columns={c: norm(c) for c in df.columns})

COL_ALIASES = {
    "community_area": [
        "community_area","community","area","area_id","comm_area",
        "area_num","area_number","area_numbe","community_area_id",
        "communityarea","community_area_number"
    ],
    "fp_rate": [
        "fp_rate","false_positive_rate","fp","fpr","fp_pct","fp_percent","fp_rate_mean"
    ],
    "fn_rate": [
        "fn_rate","false_negative_rate","fn","fnr","fn_pct","fn_percent","fn_rate_mean"
    ],
    "n_obs": [
        "n_obs","n","count","support","n_test","nrows","n_samples","num_obs"
    ],
    "arrest_rate_true": [
        "arrest_rate_true", "arrestrate_true", "arrest_rate_obs",
        "arrest_rate_actual", "true_arrest_rate", "arrest_rate"
    ],
}

def _resolve_col(df: pd.DataFrame, logical_name: str, required: bool=True):
    for cand in COL_ALIASES[logical_name]:
        if cand in df.columns:
            return cand
    if required:
        raise ValueError(f"Could not find a column for '{logical_name}'. "
                         f"Available columns: {list(df.columns)[:20]}")
    return None

def read_and_label(csv_path: str, label: str) -> pd.DataFrame:
    df_raw = pd.read_csv(csv_path)
    df = _normalize_columns(df_raw)

    ca_col = _resolve_col(df, "community_area", required=True)
    fp_col = _resolve_col(df, "fp_rate", required=False)
    fn_col = _resolve_col(df, "fn_rate", required=False)
    n_col  = _resolve_col(df, "n_obs",  required=False)
    ar_true_col = _resolve_col(df, "arrest_rate_true", required=False)

    out = pd.DataFrame()
    out["community_area"] = pd.to_numeric(df[ca_col], errors="coerce").astype("Int64")
    if fp_col is not None:      out["FP_rate"]          = pd.to_numeric(df[fp_col], errors="coerce")
    if fn_col is not None:      out["FN_rate"]          = pd.to_numeric(df[fn_col], errors="coerce")
    if n_col  is not None:      out["n_obs"]            = pd.to_numeric(df[n_col],  errors="coerce")
    if ar_true_col is not None: out["Arrest_Rate_true"] = pd.to_numeric(df[ar_true_col], errors="coerce")
    out["ModelType"] = label

    return out

frames = [read_and_label(p, label) for p, label in MODEL_LABELS.items()]
combined = pd.concat(frames, ignore_index=True)

combined_out = os.path.join(OUT_DIR, "combined_by_community_metrics.csv")
combined.to_csv(combined_out, index=False)
print(f"[saved combined metrics] {os.path.abspath(combined_out)}")

# load shapefile
gdf_geo = gpd.read_file(COMMUNITY_SHP)
if gdf_geo.crs is None or gdf_geo.crs.to_epsg() != 4326:
    gdf_geo = gdf_geo.to_crs(epsg=4326)

CAND_KEYS = ["area_numbe","area_num_1","area_num","AREA_NUMBE","AREA_NUM_1","AREA_NUM"]
geo_cols = list(gdf_geo.columns)
left_key = next((k for k in CAND_KEYS if k in geo_cols), None)

if left_key is None:
    for col in gdf_geo.columns:
        if col.lower().startswith("area") and pd.api.types.is_numeric_dtype(gdf_geo[col]):
            try:
                if pd.to_numeric(gdf_geo[col], errors="coerce").dropna().between(1, 77).all():
                    left_key = col
                    break
            except Exception:
                pass

if left_key is None:
    raise RuntimeError(f"Couldn't find community_area ID column in shapefile. Columns: {geo_cols[:10]}")

gdf_geo[left_key] = pd.to_numeric(gdf_geo[left_key], errors="coerce").astype("Int64")
geojson_obj = json.loads(gdf_geo.to_json())

# color scaling
def metric_scale(values):
    vals = pd.Series(values).dropna()
    if len(vals) == 0:
        return 0.0, 1.0
    vmin, vmax = float(vals.min()), float(vals.max())
    if math.isclose(vmin, vmax):
        vmin -= 1e-9
        vmax += 1e-9
    return vmin, vmax

def _per_map_scale(df_model, metric):
    dfm = df_model.copy()
    if "n_obs" in dfm.columns:
        dfm = dfm[pd.to_numeric(dfm["n_obs"], errors="coerce").fillna(0) >= MIN_SUPPORT]
    series = pd.to_numeric(dfm[metric], errors="coerce") if metric in dfm.columns else pd.Series([], dtype=float)
    return metric_scale(series)

def build_lookup(df, metric):
    lookup = {}
    has_n = "n_obs" in df.columns
    for _, r in df.iterrows():
        ca = r.get("community_area", None)
        if pd.isna(ca):
            continue
        entry = {"metric": None, "n": None}
        if metric in df.columns and pd.notnull(r.get(metric, np.nan)):
            entry["metric"] = float(r[metric])
        if has_n and pd.notnull(r.get("n_obs", np.nan)):
            entry["n"] = int(r["n_obs"])
        lookup[int(ca)] = entry
    return lookup

def make_style_func(lookup, colormap):
    def _style(feat):
        ca_val = feat["properties"].get(left_key)
        try:
            ca = int(ca_val)
        except Exception:
            ca = None

        fill = "#dddddd"
        opacity = 0.3
        if ca in lookup:
            val = lookup[ca]["metric"]
            n   = lookup[ca]["n"]
            if val is not None and (n is None or n >= MIN_SUPPORT):
                fill = colormap(val)
                opacity = 0.8

        return {"fillColor": fill, "color": "#202020", "weight": 0.7, "fillOpacity": opacity}
    return _style

def _colormap(metric, vmin, vmax):
    if metric == "Arrest_Rate_true":
        return LinearColormap(
            colors=["#ffffb2", "#fecc5c", "#fd8d3c", "#f03b20", "#bd0026"],
            vmin=vmin, vmax=vmax
        )
    else:
        return LinearColormap(["blue", "red"], vmin=vmin, vmax=vmax)

def make_map(geojson_obj, df_model, metric, title, filename):
    if metric not in df_model.columns:
        print(f"[warn] Metric '{metric}' missing; skipping map '{title}'.")
        return None

    vmin, vmax = _per_map_scale(df_model, metric)
    cmap = _colormap(metric, vmin, vmax)
    cmap.caption = title

    lookup = build_lookup(df_model, metric)

    m = folium.Map(location=[41.8781, -87.6298], zoom_start=10, tiles="cartodbpositron")
    folium.GeoJson(
        data=geojson_obj,
        name=title,
        style_function=make_style_func(lookup, cmap),
        tooltip=folium.features.GeoJsonTooltip(
            fields=[left_key] + (["community"] if "community" in gdf_geo.columns else []),
            aliases=["Area #"] + (["Community"] if "community" in gdf_geo.columns else []),
            localize=True,
            sticky=False,
            labels=True
        ),
        highlight_function=lambda x: {"weight": 2, "color": "#000000"},
    ).add_to(m)

    cmap.add_to(m)
    folium.LayerControl(collapsed=True).add_to(m)

    out_html = os.path.join(OUT_DIR, filename)
    m.save(out_html)
    print(f"[saved map] {os.path.abspath(out_html)}")
    return out_html

html_paths = []

#model maps
for model_name in sorted(combined["ModelType"].unique()):
    df_m = combined[combined["ModelType"] == model_name]
    for metric in PER_MODEL_METRICS:
        p = make_map(
            geojson_obj,
            df_m,
            metric,
            title=f"{metric} — {model_name}",
            filename=f"{model_name}_{metric}.html"
        )
        if p: html_paths.append(p)

#arrest rate
if SINGLE_METRIC in combined.columns:
    ar_df = (combined[["community_area", SINGLE_METRIC, "n_obs"]]
             .dropna(subset=["community_area", SINGLE_METRIC])
             .sort_values(["community_area"])
             .drop_duplicates(subset=["community_area"]))
    p = make_map(
        geojson_obj,
        ar_df,
        SINGLE_METRIC,
        title="Arrest Rate (Observed)",
        filename="Arrest_Rate_true.html"
    )
    if p: html_paths.append(p)
else:
    print("[warn] No Arrest_Rate_true column found in any input; skipping that map.")


for p in html_paths:
    try:
        webbrowser.open(f"file://{os.path.abspath(p)}")
    except Exception:
        pass


[saved combined metrics] /Users/reesemullen/maps_by_model_metric/combined_by_community_metrics.csv
[saved map] /Users/reesemullen/maps_by_model_metric/Gradient_Boosted_Trees_FP_rate.html
[saved map] /Users/reesemullen/maps_by_model_metric/Gradient_Boosted_Trees_FN_rate.html
[saved map] /Users/reesemullen/maps_by_model_metric/Logistic_Regression_FP_rate.html
[saved map] /Users/reesemullen/maps_by_model_metric/Logistic_Regression_FN_rate.html
[saved map] /Users/reesemullen/maps_by_model_metric/Random_Forest_FP_rate.html
[saved map] /Users/reesemullen/maps_by_model_metric/Random_Forest_FN_rate.html
[saved map] /Users/reesemullen/maps_by_model_metric/Arrest_Rate_true.html
